In [2]:
#!/usr/bin/env python
# coding: utf-8

from gurobipy import *
import numpy as np
import pandas as pd 
import numpy as np
os.chdir("C:\\Users\\uqrbean1\\OneDrive - The University of Queensland\\Documents\\ieee")

price = np.loadtxt("price.flat")
netload = np.loadtxt("ranger2.flat") 
        
## only do small -- 5 hours

#flist = glob.glob("phase2_instance_small_0.txt")

flist = [    
    "phase2_instance_small_0.txt",
    "phase2_instance_small_1.txt",
    "phase2_instance_small_2.txt",
    "phase2_instance_small_3.txt",
    "phase2_instance_small_4.txt",
    "phase2_instance_large_0.txt",
    "phase2_instance_large_1.txt",
    "phase2_instance_large_2.txt",
    "phase2_instance_large_3.txt",
    "phase2_instance_large_4.txt"    
        ]

for file in flist:
    count_data = np.genfromtxt(file,delimiter=' ',max_rows=1,usecols=(1,2,3,4,5),dtype=int)
    building_count = count_data[0]
    solar_count = count_data[1]
    battery_count = count_data[2]
    recurring_count = count_data[3]
    once_count = count_data[4]
    building_data = np.genfromtxt(file,delimiter=' ',skip_header=1,max_rows=building_count,usecols=(1,2,3),dtype=int)
    building_num = building_data[:,0]
    building_small = building_data[:,1]
    building_large = building_data[:,2]
    battery_data = np.genfromtxt(file,delimiter=' ',skip_header=1+solar_count+building_count,max_rows=battery_count,
                                 usecols=(1,3,4,5),dtype=float)

    #array([[  0.  , 150.  ,  75.  ,   0.85],
    #       [  1.  , 420.  ,  60.  ,   0.6 ]])

    # now variable columns

    with open(file, 'r') as f:
        file_lines = [line.rstrip() for line in f]

    df =  pd.DataFrame([string.split(' ') for string in file_lines])
    alldata = np.array(df)

    recur_data = alldata[alldata[:,0]=='r',]
    once_data = alldata[alldata[:,0]=='a',]

    once_data = np.array(df)

    recur_small = recur_data[recur_data[:,3]=='S']
    recur_large = recur_data[recur_data[:,3]=='L']

    T               = 2880                                      # number of time periods
    timeperiods     = range(T)     
    TX = 2881
    timeperiodsx = range(TX)
    
    T1 = 532
    timeperiods1 = range(T1)
    
    nBuildings = building_count 
    buildings = range(nBuildings)
    
    B = 2
    batteries = range(B)

    smallActIdx = pd.to_numeric(recur_small[:,1])
    largeActIdx = pd.to_numeric(recur_large[:,1])

    smallAct = range(smallActIdx.shape[0])
    largeAct = range(largeActIdx.shape[0])

    ActEnergy = pd.to_numeric(recur_data[:,4])

    smallActPeriods = recur_small[:,5]
    largeActPeriods = recur_large[:,5]
    ActPeriods = pd.to_numeric(recur_data[:,5])

    ActPrec = pd.to_numeric(recur_data[:,6])

    ActRooms = pd.to_numeric(recur_data[:,2])

    Act = range(recurring_count)
    
    bat_cap = [150, 420]   

    week1 = list(range(88,120)) + list(range(184,216)) + list(range(280,312)) + list(range(376,408)) + list(range(472,504))
    week2 = list(range(88+672,120+672)) + list(range(184+672,216+672)) + list(range(280+672,312+672)) + list(range(376+672,408+672)) + list(range(472+672,504+672))
    week3 = list(range(88+672*2,120+672*2)) + list(range(184+672*2,216+672*2)) + list(range(280+672*2,312+672*2)) + list(range(376+672*2,408+672*2)) + list(range(472+672*2,504+672*2))
    week4 = list(range(88+672*3,120+672*3)) + list(range(184+672*3,216+672*3)) + list(range(280+672*3,312+672*3)) + list(range(376+672*3,408+672*3)) + list(range(472+672*3,504+672*3))
    #week5 = list(range(88+672*4,120+672*4)) + list(range(184+672*4,2880)) # Monday 30 Nov and 8 periods of Tuesday 1 Dec
   
    #weeks = week1 + week2 + week3 + week4 + week5
    weeks = week1 + week2 + week3 + week4
    
    notweek = list(set(timeperiods) - set(weeks))
    notweek1 = list(set(timeperiods) - set(week1)) 
        
    m = Model("monash_schedule")

    ## for every Activity ... 0 to 49 or 0 to 199 ...
    ## the "bu" matrix reflects activities * buildings -- can't mix small and large buildings

    w            = m.addVars(Act, timeperiods, lb=-1, ub=1, name="w") # intermediate variable for differences
    v            = m.addVars(Act, timeperiods, name="v") # max v 
    s            = m.addVars(Act, timeperiods, vtype=GRB.BINARY, name="s") # binary allocation variable

    bu           = m.addVars(Act, buildings, lb=0, ub=3, vtype=GRB.INTEGER, name="roomlimit") # small_ac_buildings - sum to activity room count
    small_use       = m.addVars(timeperiods, buildings, lb=0, name="small_use")
    large_use       = m.addVars(timeperiods, buildings, lb=0, name="large_use")

    actLoad      = m.addVars(Act, timeperiods, name="load")
    load         = m.addVars(timeperiods, name="load1") # a variable load -- add net load and activity load and batteries 
    maxload      = m.addVar(name="maxload")
    maxrecurload = m.addVar(name="maxrecurload")
    obj_lin      = m.addVar(name="obj_lin")
    recurload = m.addVars(timeperiods, name="recurload")
    
    disch  =   m.addVars(batteries, timeperiods, name="batd", vtype=GRB.BINARY) # binary decision - ch or disch at time t, 0 to T-1
    ch    =    m.addVars(batteries, timeperiods, name="batc", vtype=GRB.BINARY)
    batlevel = m.addVars(batteries,timeperiodsx, name="batlevel", lb=0) ## 0 to T-1 and T itself 
    charge_effect = m.addVars(timeperiods, name="ch_effect", lb=-GRB.INFINITY ) ## the net charge effect to calculate load1 then maxload 
    
    #guess_max_netload = 744.33 ## guess max load for October 2020
            
    ## get the start hours - in the derived variable "z"

    z  = m.addVars(Act, lb=0,ub=T1-1, name="z")
    z0 = m.addVars(Act, timeperiods1, name="z0")
    
    for a in Act:
        m.addConstrs((z0[(a,t)] == t*v[(a,t)]) for t in timeperiods1)

    for a in Act:
        m.addConstr(z[a] == max_(z0[a,t] for t in timeperiods1))

    constr = m.addConstr((obj_lin == quicksum([(load[t] * price[t]) for t in timeperiods])/4000) , "linear_objective")
    m.setObjective(obj_lin + maxload*maxload/200, GRB.MINIMIZE)
        
    constr = m.addConstrs((load[t] == (netload[t] + charge_effect[t] + recurload[t]) for t in week1), name="netload1")
    constr = m.addConstrs((load[t] == (netload[t] + charge_effect[t] + recurload[t-672]) for t in week2), name="netload2")
    constr = m.addConstrs((load[t] == (netload[t] + charge_effect[t] + recurload[t-672*2]) for t in week3), name="netload3")
    constr = m.addConstrs((load[t] == (netload[t] + charge_effect[t] + recurload[t-672*3]) for t in week4), name="netload4")
    #constr = m.addConstrs((load[t] == (netload[t] + charge_effect[t] + recurload[t-672*4]) for t in week5), name="netloadx")
    constr = m.addConstrs((load[t] == (netload[t] + charge_effect[t]) for t in notweek), name="netload5")

    ## constraint s, the binary allocation variable -- activities must occur in blocks, and z calculates the start hour 

    constr = m.addConstrs((w[(a,t)] == (s[(a,t)] - s[(a,t-1)]) for a in Act for t in range(1,T1)), name="shift_starts_hat")
    constr = m.addConstrs((v[(a,t)] == max_(w[(a,t)], 0.0) for a in Act for t in range(1,T1)), name="shift_starts")
    constr = m.addConstrs((v[(a,0)] == s[(a,0)] for a in Act), name="shift_starts_init")
    constr = m.addConstrs((quicksum([v[(a,t)] for t in timeperiods1]) <= 1 for a in Act), name="shift_start_max")

    ## maxload is the load over all time periods 
    constr = m.addConstrs((recurload[t] == (quicksum([actLoad[(a,t)] for a in Act])) for t in week1), name="netloada")
    
    ## should have been for t in weeks in this line
    
    constr = m.addConstr(maxload == max_([load[t] for t in weeks]),name="get_max_load") ## for the objective function
    constr = m.addConstr(maxrecurload == max_([recurload[t] for t in week1]),name="get_maxr_load1")

    ## the sum down timeperiods of s must equal the number of periods required for that activity

    constr = m.addConstrs((quicksum([s[(a,t)] for t in week1]) == ActPeriods[a] for a in Act), name="col")

    #constr = m.addConstrs((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t)] for a in smallActIdx]) for b in buildings for t in timeperiods ),name="bus")

    #constr = m.addConstrs((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t)] for a in largeActIdx]) for b in buildings for t in timeperiods ),name="bul")
    
    for b in buildings:
        for t in week1:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t)] for a in smallActIdx])),name="buso1")
            
    for b in buildings:
        for t in week2:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672)] for a in smallActIdx])),name="buso2")

    for b in buildings:
        for t in week3:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*2)] for a in smallActIdx])),name="buso3")

    for b in buildings:
        for t in week4:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*3)] for a in smallActIdx])),name="buso4")
    
    #for b in buildings:
    #    for t in week5:
    #        constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*4)] for a in smallActIdx])),name="buso5")

    for b in buildings:
        for t in week1:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t)] for a in largeActIdx])),name="bulo1")
            
    for b in buildings:
        for t in week2:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672)] for a in largeActIdx])),name="bulo2")

    for b in buildings:
        for t in week3:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*2)] for a in largeActIdx])),name="bulo3")

    for b in buildings:
        for t in week4:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*3)] for a in largeActIdx])),name="bulo4")
        
    #for b in buildings:
    #    for t in week5:
    #        constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*4)] for a in largeActIdx])),name="bulo5") 

    constr = m.addConstrs((small_use[(t,b)] <= building_small[b] for t in timeperiods for b in buildings), name="small_lim")
    constr = m.addConstrs((large_use[(t,b)] <= building_large[b] for t in timeperiods for b in buildings), name="large_lim")

    const = m.addConstrs((quicksum([bu[(a,b)] for b in buildings]) == ActRooms[a] for a in smallActIdx ),name="smallrooms")
    const = m.addConstrs((quicksum([bu[(a,b)] for b in buildings]) == ActRooms[a] for a in largeActIdx ),name="largerooms")

    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(0,88)]) == 0 for a in Act),name="zero1")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(120,184)]) == 0 for a in Act),name="zero2")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(216,280)]) == 0 for a in Act),name="zero3")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(312,376)]) == 0 for a in Act),name="zero4")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(408,472)]) == 0 for a in Act),name="zero5")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(504,2880)]) == 0 for a in Act),name="zero6")
    for a in Act:
        ro = pd.to_numeric(recur_data[a,6:])
        co = int(ro[0])
        for i in range(co):
            m.addConstr((z[a] - z[int(ro[i+1])] >= 65),name="prec") ## has to be on a previous day -- let's say at least 16 hours ago

    constr = m.addConstrs((actLoad[(a,t)] == ActEnergy[a]*ActRooms[a]*s[(a,t)]) for a in Act for t in week1)
    
    #########################
    
    constr = m.addConstrs((disch[b,t] + ch[b,t] <= 1 for b in batteries for t in timeperiods), name="batres")

    ## do what cha like, but make sure for all t NETLOAD[t] + CHARGE_EFFECT[t] <= MAX(NETLOAD(T))
    ## perhaps experiment with removing this if I like living on the edge?????

    #constr = m.addConstrs((recurload[t] + charge_effect[t] <= maxrecurload for t in week1), name="batmaxlimit1")
    #constr = m.addConstrs((recurload[t-672] + charge_effect[t] <= maxrecurload for t in week2), name="batmaxlimit2")
    #constr = m.addConstrs((recurload[t-672*2] + charge_effect[t] <= maxrecurload for t in week3), name="batmaxlimit3")
    #constr = m.addConstrs((recurload[t-672*3] + charge_effect[t] <= maxrecurload for t in week4), name="batmaxlimit4")
    #constr = m.addConstrs((recurload[t-672*4] + charge_effect[t] <= maxrecurload for t in week5), name="batmaxlimit5")
    
    constr = m.addConstrs((ch[b,t] == 0 for b in batteries for t in weeks), name="batpeak1")
    #constr = m.addConstrs((disch[0,t] + disch[1,t] >= 1 for t in weeks), name="batpeak2") # one battery is disch

    constr = m.addConstrs((batlevel[b,0] == bat_cap[b] for b in batteries), name="batstart")
    #constr = m.addConstr((mdis == max_([disch[0,t] for t in timeperiods])), name="batend") ## must discharge battery 0 at some point 
    #constr = m.addConstr(mdis > 0.0,name="md0")

    bat_rate = [ 75, 60 ] 
    bat_ch    = [ 75/math.sqrt(0.85), 60/math.sqrt(0.60) ] # charging battery by 75 actually takes 81 kW, disch by 75 sends 69 
    bat_disch = [ 75*math.sqrt(0.85), 60*math.sqrt(0.60) ] 

    constr = m.addConstrs((batlevel[b,t+1] == (batlevel[b,t] - bat_rate[b]*disch[b,t]/4 + bat_rate[b]*ch[b,t]/4 ) ##
                          for t in timeperiods for b in batteries ), name="batlev")

    constr = m.addConstrs((batlevel[b,t] <= bat_cap[b] for t in timeperiodsx for b in batteries),name="batcap")

    constr = m.addConstrs((charge_effect[t] == quicksum([bat_ch[b] * ch[b,t] - bat_disch[b] * disch[b,t] for b in batteries]) 
                          for t in timeperiods), name="charge_effect")
    #################################     
        
    sfile = file.replace("phase2","p2sol\\") # the given start point
    sfile2 = sfile.replace("_instance_","") # the given start point
    sfile3 = sfile2.replace("_","") # the given start point
    sfile4 = sfile3.replace("txt","mps") # the given start point
    m.write(sfile4)
        
    #constr = m.addConstr(((maxrecurload+583)*(maxrecurload+583)-1257*1257) / 200 <= obj_lin + maxload*maxload/200 - 26985)
    
    #m.read("s45.sol")

Academic license - for non-commercial use only - expires 2022-09-07
Using license file C:\Users\uqrbean1\gurobi.lic


Academic license - for non-commercial use only - expires 2022-09-07
Using license file C:\Users\uqrbean1\gurobi.lic


In [ ]:
m.update()
m.read("s45.sol")

In [2]:
m.Params.timelimit = 4*3600.0

Changed value of parameter timelimit to 14400.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [3]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 120874 rows, 663435 columns and 428123 nonzeros
Model fingerprint: 0xca922252
Model has 1 quadratic objective term
Model has 8161 quadratic constraints
Model has 26602 general constraints
Variable types: 507615 continuous, 155820 integer (155520 binary)
Coefficient statistics:
  Matrix range     [2e-05, 6e+02]
  QMatrix range    [5e-03, 1e+00]
  QLMatrix range   [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [1e-02, 1e-02]
  Bounds range     [1e+00, 5e+02]
  RHS range        [1e+00, 7e+02]
  QRHS range       [3e+04, 3e+04]

User MIP start produced solution with objective 26993.1 (1.18s)
Loaded user MIP start with objective 26993.1
Processed MIP start in 1.20 seconds

Presolve removed 79734 rows and 617473 columns
Presolve time: 3.99s
Presolved: 123620 rows, 72977 columns, 356577 nonzeros
Presolved model has 720 S

In [4]:
m.write("s45-testing.sol")

In [ ]:
m.optimize()